In [2]:
import pandas as pd
import numpy as np
import time
import sys
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import json
from src.utils.utils import fetch_path_from_config
from src.constants.constants import BASE_DIR,CONFIG_PATH

In [4]:
data_path = fetch_path_from_config("Paths", "data_path", CONFIG_PATH)

raw_df = pd.read_csv(data_path, index_col=0)
raw_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",date_time,Activity
0,0.229771,0.006191,-0.063642,0.148432,-0.017294,-0.214122,0.109120,-0.140795,-0.060678,0.136123,...,-0.669427,0.001615,-0.503460,-0.695799,0.618240,-0.487791,0.006897,0.012113,2020-01-01 01:00:00,WALKING_DOWNSTAIRS
1,0.096259,-0.004479,-0.011539,-0.628955,-0.803592,-0.217212,-0.174894,-0.421507,-0.025912,-0.539172,...,-0.286686,-0.079703,0.052752,0.000921,-0.014742,0.155335,-0.236586,-0.277595,2020-01-01 01:01:00,LAYING
2,0.034372,-0.043343,-0.311560,-0.075527,-0.063103,0.025375,-0.032692,-0.083888,0.127356,-0.021718,...,-0.136133,0.005180,0.202458,0.084711,-0.297366,-0.540108,0.063798,0.131656,2020-01-01 01:02:00,WALKING_UPSTAIRS
3,0.024097,0.009133,-0.086416,0.071118,0.006441,-0.122816,-0.032050,0.024451,-0.164227,0.075734,...,-0.206189,0.047049,0.292424,0.064996,0.100568,-0.682927,0.130937,0.021554,2020-01-01 01:03:00,WALKING_DOWNSTAIRS
4,0.225618,-0.013047,-0.103685,-0.189633,-0.674050,-0.602043,-0.820340,-0.008993,-0.988655,-0.698891,...,-0.843761,0.008899,-0.189814,-0.075739,-0.130241,-0.059553,-0.011284,-0.077751,2020-01-01 01:04:00,SITTING


In [ ]:
latest_record = None

def generate_data(iter_index):
    global latest_record
    while True:
        result_data =  raw_df.loc[next(iter_index)].to_json()
        latest_record = result_data
        yield f"{result_data}\n"
        time.sleep(60)

iter_index = iter(raw_df.index.to_list())


In [5]:
app = FastAPI()

data_stream = None


@app.get("/generate")
def start_api():

    global data_stream

    if data_stream is None:
        data_stream = generate_data(iter_index)
    return StreamingResponse(data_stream, media_type="application/json")

@app.get("/stop")
def stop_data_generation():
    """Stop real-time data generation."""
    global data_stream
    data_stream = None
    return {"message": "Data generation stopped"}
    
@app.get("/")
def read_root():
    """API Root Endpoint."""
    return {"message": "Welcome to the Real-Time Data API"}
        
@app.get("/latest")
def get_latest_record():
    global latest_record
    if latest_record:
        return json.loads(latest_record)  # Return the latest record as a JSON response
    else:
        return {"message": "No data has been generated yet."}

In [24]:
import requests


response = requests.get("http://127.0.0.1:8888/latest")

res = response.json()


In [25]:
res

{'tBodyAcc-mean()-X': 0.0811780436,
 'tBodyAcc-mean()-Y': -0.0214063116,
 'tBodyAcc-mean()-Z': -0.0459687021,
 'tBodyAcc-std()-X': -0.0322820913,
 'tBodyAcc-std()-Y': 0.0247647348,
 'tBodyAcc-std()-Z': -0.0052815106,
 'tBodyAcc-mad()-X': -0.0725876259,
 'tBodyAcc-mad()-Y': 0.0789431566,
 'tBodyAcc-mad()-Z': -0.0885296332,
 'tBodyAcc-max()-X': -0.1648442056,
 'tBodyAcc-max()-Y': -0.0329830166,
 'tBodyAcc-max()-Z': -0.134905976,
 'tBodyAcc-min()-X': 0.0830972856,
 'tBodyAcc-min()-Y': -0.003583581,
 'tBodyAcc-min()-Z': 0.525799039,
 'tBodyAcc-sma()': -0.0083387655,
 'tBodyAcc-energy()-X': -0.2830078154,
 'tBodyAcc-energy()-Y': -0.4978098345,
 'tBodyAcc-energy()-Z': -0.0205639854,
 'tBodyAcc-iqr()-X': -0.2340017874,
 'tBodyAcc-iqr()-Y': 0.0040886087,
 'tBodyAcc-iqr()-Z': -0.5670431165,
 'tBodyAcc-entropy()-X': 0.2613722153,
 'tBodyAcc-entropy()-Y': 0.191366209,
 'tBodyAcc-entropy()-Z': -0.0772591118,
 'tBodyAcc-arCoeff()-X,1': -0.4402935542,
 'tBodyAcc-arCoeff()-X,2': 0.2344995801,
 'tBody

In [31]:
import pandas as pd

df = pd.DataFrame(data = res, index=[1])
inference_data = df.drop(['Activity','date_time'], axis=1)
ground_truth = df['Activity']

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",date_time,Activity
1,0.081178,-0.021406,-0.045969,-0.032282,0.024765,-0.005282,-0.072588,0.078943,-0.08853,-0.164844,...,-0.143462,0.195898,0.219257,0.576794,0.203269,-0.567947,0.065989,0.005072,2020-01-01 01:17:00,WALKING_UPSTAIRS


In [18]:
from src.constants.constants import CONFIG_PATH
from src.utils.utils import fetch_path_from_config
import pandas as pd
import configparser

fetch_path_from_config("Paths", "model_registery_path", CONFIG_PATH)

'E:\\Learning\\BlackMi_mobiles_v_0\\model_registery\\model_registery_ver_02'